In [21]:
# Model design
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
from IPython.display import HTML

# Spawn points
spawns = ((16,1),(15,18),(30,10),(1,9))

stoplights = ((14,8),(14,11),(17,11),(17,8))

roadup = ((16,1),(16,2),(16,3),(16,4),(16,5),(16,6),(16,7),(16,8),(16,9),(16,10),(16,11),(16,12),(16,13),(16,14),(16,15))
roaddown = ((15,1),(15,2),(15,3),(15,4),(15,5),(15,6),(15,7),(15,8),(15,9),(15,10),(15,11),(15,12),(15,13),(15,14),(15,15),(15,16),(15,17),(15,18))
roadleft = ((1,10),(2,10),(3,10),(4,10),(5,10),(6,10),(7,10),(8,10),(9,10),(10,10),(11,10),(12,10),(13,10),(14,10),(15,10),(16,10),(17,10),(18,10),(19,10),(20,10),(21,10),(22,10),(23,10),(24,10),(25,10),(26,10),(27,10),(28,10),(29,10),(30,10))
roadright = ((1,9),(2,9),(3,9),(4,9),(5,9),(6,9),(7,9),(8,9),(9,9),(10,9),(11,9),(12,9),(13,9),(14,9),(15,9),(16,9),(17,9),(18,9),(19,9),(20,9),(21,9),(22,9),(23,9),(24,9),(25,9),(26,9),(27,9),(28,9),(29,9),(30,9))

# Direct Routes
route1 = ((16,1), (16,8), (16,9), (16,18))       # Up
route2 = ((15,18), (15,11), (15,10), (15,1))     # Down
route3 = ((30,10), (17,10), (16,10), (1,10))     # Left
route4 = ((1,9), (14,9), (15,9), (30,9))         # Right

routes = (route1, route2, route3, route4)


In [48]:
class Car(ap.Agent):
    
    def setup(self):
        # Initialize an attribute with a parameter
        self.grid = self.model.grid
        self.random = self.model.random
        self.route = self.random.choice(routes)
        print(self.route)

        # Spawn the car in random from list of spawn points
        # Car must have objetive
        # Tracks that the objetive was reached, 0 = not reached, 1 = reached
        self.goal = 0
        self.group = 1

        # Agent position in x and y
        self.x = self.route[0][0]
        self.y = self.route[0][1]

        # Agent objective in x and y
        self.objx = self.route[3][0]
        self.objy = self.route[3][1]
        
        # Agent in movement or stopped, 0 = stopped, 1 = moving
        self.moving = 0

        # Step that Agent is at
        self.actStep = 1

        self.xstep = 1
        self.ystep = 1

        self.position = (self.x, self.y)

    def check_goal(self):
        if self.x == self.objx and self.y == self.objy:
            self.goal = 1
        else:
            self.goal = 0
            
    # Agent's Movement
    def moveXright(self):
        self.x += self.xstep
        self.grid.move_to(self, (self.x,self.y))
        self.actStep += 1

    def moveXleft(self):
        self.x -= self.xstep
        self.grid.move_to(self, (self.x,self.y))
        self.actStep += 1

    def moveYup(self):
        self.y += self.ystep
        self.grid.move_to(self, (self.x,self.y))
        self.actStep += 1

    def moveYdown(self):
        self.y -= self.ystep
        self.grid.move_to(self, (self.x,self.y))
        self.actStep += 1

    def move(self):
        if (self.x != self.objx):
            if (self.x > self.objx):
                self.moveXleft()
            else:
                self.moveXright()
        else:
            if (self.y > self.objy):
                self.moveYdown()
            else:
                self.moveYup()
        print("Position at ", self.x, " and ", self.y)

    # Must have method to react to its surroundings (Stoplight)


In [49]:
class Traffic(ap.Model):

    def setup(self):
        n = self.p.cars
        
        self.grid = ap.Grid(self, (self.p.x, self.p.y), track_empty=True)
        # Initiate car agents on to the grid
        self.agents = ap.AgentList(self, n, Car)
        self.grid.add_agents(self.agents, random=False, empty=False)


    def step(self):
        self.cars_on_road.move()
        # Stop simulation if all cars reached objective


    def update(self):
        self.agents.check_goal()

        self.cars_on_road = self.agents.select(self.agents.goal == 0)
        # Executed after every step done
        if len(self.cars_on_road) == 0:
           self.stop()


    def end(self):
        # Called at the end of the simulation
        
        self.report('Simulation ran', 1)

In [50]:
# Define parameters
parameters = {
    'steps' : 30,
    'cars' : 4, # Amount of cars
    'x' : 30, # Width of grid
    'y' : 18, # Height of grid
}

In [51]:
model = Traffic(parameters)
print(model)
results = model.run()

Traffic
((30, 10), (17, 10), (16, 10), (1, 10))
((16, 1), (16, 8), (16, 9), (16, 18))
((30, 10), (17, 10), (16, 10), (1, 10))
((30, 10), (17, 10), (16, 10), (1, 10))
calling goal func
Checking goal
Checking goal
Checking goal
Checking goal
Position at  29  and  10
Position at  16  and  2
Position at  29  and  10
Position at  29  and  10
calling goal func
Checking goal
Checking goal
Checking goal
Checking goal
Completed: 1 stepsPosition at  28  and  10
Position at  16  and  3
Position at  28  and  10
Position at  28  and  10
calling goal func
Checking goal
Checking goal
Checking goal
Checking goal
Completed: 2 stepsPosition at  27  and  10
Position at  16  and  4
Position at  27  and  10
Position at  27  and  10
calling goal func
Checking goal
Checking goal
Checking goal
Checking goal
Completed: 3 stepsPosition at  26  and  10
Position at  16  and  5
Position at  26  and  10
Position at  26  and  10
calling goal func
Checking goal
Checking goal
Checking goal
Checking goal
Completed: 4 s

In [57]:
def animation_plot(model, ax):
    group_grid = model.grid.attr_grid('group')
    ap.gridplot(group_grid, cmap='Accent', ax=ax)
    ax.set_title(f"Traffic Simulation")

fig, ax = plt.subplots() 
model = Traffic(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())

((30, 10), (17, 10), (16, 10), (1, 10))
((15, 18), (15, 11), (15, 10), (15, 1))
((30, 10), (17, 10), (16, 10), (1, 10))
((16, 1), (16, 8), (16, 9), (16, 18))
calling goal func
Checking goal
Checking goal
Checking goal
Checking goal
Position at  29  and  10
Position at  15  and  17
Position at  29  and  10
Position at  16  and  2
calling goal func
Checking goal
Checking goal
Checking goal
Checking goal
Position at  28  and  10
Position at  15  and  16
Position at  28  and  10
Position at  16  and  3
calling goal func
Checking goal
Checking goal
Checking goal
Checking goal
Position at  27  and  10
Position at  15  and  15
Position at  27  and  10
Position at  16  and  4
calling goal func
Checking goal
Checking goal
Checking goal
Checking goal
Position at  26  and  10
Position at  15  and  14
Position at  26  and  10
Position at  16  and  5
calling goal func
Checking goal
Checking goal
Checking goal
Checking goal
Position at  25  and  10
Position at  15  and  13
Position at  25  and  10
P